# VKcup 2022

In [203]:
# %pip install tensorflow_text==2.2.0
# %pip install tensorflow==2.2.0

In [204]:
import pandas as pd
import numpy as np
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set()

In [205]:
train = pd.read_csv('train.csv')


### Посмотим на датасет

In [206]:
train.head(10)

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
5,780575952,boardgames,Северные территории на границе королевства Нал...
6,741108447,hockey,Рашит Давыдов поделился деталями общения с гол...
7,642761184,esport,Какой т3 игрок на данный момент перспективнее ...
8,418053478,athletics,7 августа в Москве в Измайловском парке состои...
9,748701390,motosport,Ребята подскажите диску капец или нет Таких тр...


In [207]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38740 entries, 0 to 38739
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       38740 non-null  int64 
 1   category  38740 non-null  object
 2   text      38740 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.1+ KB


In [208]:
data = train[['category', 'text']]

In [209]:
data.head(3)

,category,text
0,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,football,Лучшие чешские вратари – Доминик Доминатор Гаш...


Список с метками классов. Метки переведем в числовой вид


In [210]:
labels = {'athletics': 0,
          'autosport': 1,
          'basketball': 2,
          'boardgames': 3,
          'esport': 4,
          'extreme': 5,
          'football': 6,
          'hockey': 7,
          'martial_arts': 8,
          'motosport': 9,
          'tennis': 10,
          'volleyball': 11,
          'winter_sport': 12}

data['category'] = data['category'].map(labels)
  
data.head()


C:\Users\denis\AppData\Local\Temp\ipykernel_13180\1909585490.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['category'] = data['category'].map(labels)


,category,text
0,12,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,5,Возвращение в подземелье Треша 33 Эйфория тупо...
2,6,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,3,Rtokenoid Warhammer40k валрак решил нас подкор...
4,7,Шестеркин затаскивает Рейнджерс в финал Восточ...


split tran

In [211]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [212]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [213]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [214]:
y = tf.keras.utils.to_categorical(data['category'].values, num_classes=len(labels))

x_train, x_test, y_train, y_test = train_test_split(data['text'], y, test_size=0.25)

loading BERT using tfHub

In [215]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
  '''return BERT-like embeddings of input text
  Args:
    - sentences: list of strings
  Output:
    - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
  '''
  preprocessed_text = preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']


In [216]:
from tensorflow.keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [217]:
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(len(labels), activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [218]:
from keras.models import load_model


n_epochs = 10

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      balanced_recall,
      balanced_precision,
      balanced_f1_score
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)

model_fit = model.fit(x_train, 
                      y_train, 
                      epochs = n_epochs,
                      validation_data = (x_test, y_test),
                      callbacks = [earlystop_callback])



Epoch 1/10
908/908 [==============================] - 6675s 7s/step - loss: 1.3819 - accuracy: 0.6023 - balanced_recall: 0.2811 - balanced_precision: 0.4413 - balanced_f1_score: 0.3406 - val_loss: 1.0188 - val_accuracy: 0.7123 - val_balanced_recall: 0.4764 - val_balanced_precision: 0.6654 - val_balanced_f1_score: 0.5533
Epoch 2/10
908/908 [==============================] - 6608s 7s/step - loss: 0.9862 - accuracy: 0.7088 - balanced_recall: 0.5072 - balanced_precision: 0.6827 - balanced_f1_score: 0.5799 - val_loss: 0.9245 - val_accuracy: 0.7299 - val_balanced_recall: 0.5485 - val_balanced_precision: 0.7116 - val_balanced_f1_score: 0.6176
Epoch 3/10
908/908 [==============================] - 6391s 7s/step - loss: 0.9198 - accuracy: 0.7250 - balanced_recall: 0.5561 - balanced_precision: 0.7091 - balanced_f1_score: 0.6217 - val_loss: 0.8786 - val_accuracy: 0.7426 - val_balanced_recall: 0.5749 - val_balanced_precision: 0.7212 - val_balanced_f1_score: 0.6382
Epoch 4/10
908/908 [==============

time to predict

In [219]:
def predict_class(reviews):
  '''predict class of input text
  Args:
    - reviews (list of strings)
  Output:
    - class (list of int)
  '''
  return [np.argmax(pred) for pred in model.predict(reviews)]




loading test data

In [237]:
test = pd.read_csv('test.csv')

In [238]:
import re

test.text = test.text.apply(lambda x: x.lower())
test.text = test.text.apply(lambda x: re.sub('[.,?!-]', ' ', x))

In [239]:
from sklearn.metrics import classification_report


y_pred = predict_class(test["text"])
# print(classification_report(test["category"], y_pred))

821/821 [==============================] - 5527s 7s/step


In [222]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [240]:
y_category = list()
for pred in y_pred:
  y_category.append(get_key(labels, pred))

y_category[0:10]

['extreme',
 'martial_arts',
 'winter_sport',
 'boardgames',
 'tennis',
 'volleyball',
 'volleyball',
 'volleyball',
 'boardgames',
 'athletics']

In [241]:
submission = pd.DataFrame({
    "oid":test['oid'], 
    "category":y_category
})


In [230]:
model.save('vkcup_model')

INFO:tensorflow:Assets written to: vkcup_model\assets


INFO:tensorflow:Assets written to: vkcup_model\assets


In [242]:
submission[submission['oid']==749208109]

,oid,category
0,749208109,extreme
1687,749208109,martial_arts
2629,749208109,martial_arts
4543,749208109,martial_arts
14055,749208109,martial_arts
16139,749208109,esport
19815,749208109,esport
22985,749208109,martial_arts
24928,749208109,tennis
25046,749208109,esport


In [243]:
u_oid = submission.oid.unique()

for id in u_oid:
    mode = submission['category'][submission['oid'] == id].mode()[0]
    submission.loc[(submission['oid']==id), 'category'] = mode
    print(id, mode)





749208109 martial_arts
452466036 martial_arts
161038103 tennis
663621910 esport
566255305 tennis
332983824 basketball
306441436 autosport
770457906 volleyball
720955404 boardgames
950885012 athletics
623907520 athletics
357676832 esport
463889624 boardgames
769116238 tennis
902512702 winter_sport
573984210 hockey
867549202 boardgames
469446016 football
517149005 basketball
631811222 tennis
289454822 volleyball
633244217 winter_sport
54310919 volleyball
731046922 athletics
759095431 autosport
23592715 winter_sport
568705033 hockey
746261621 boardgames
223960908 hockey
343811314 martial_arts
39021409 motosport
433430409 winter_sport
191288802 extreme
773806324 winter_sport
42587423 autosport
909050028 volleyball
381764623 esport
896686111 athletics
759592926 winter_sport
475206109 volleyball
123458214 winter_sport
733364110 autosport
571093038 volleyball
86508807 tennis
982942625 athletics
150043234 autosport
194604309 basketball
142857803 extreme
53073826 volleyball
444234900 tennis
735

In [244]:
submission

,oid,category
0,749208109,martial_arts
1,452466036,martial_arts
2,161038103,tennis
3,663621910,esport
4,566255305,tennis
...,...,...
26255,169728316,boardgames
26256,279369911,volleyball
26257,600699419,boardgames
26258,560223506,athletics


In [245]:
submission.to_csv('submission.csv', index=False)